In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

SEED = 515
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Convolutional Sentiment Analysis
This notebook follows this tutorial: https://github.com/bentrevett/pytorch-sentiment-analysis.

In [2]:
import torchtext
from torchtext.data import Field, LabelField, BucketIterator

# Set `batch_first=True` in the `Field`.
TEXT = Field(tokenize='spacy', include_lengths=True, batch_first=True)
LABEL = LabelField(dtype=torch.float)

train_data, test_data = torchtext.datasets.IMDB.splits(TEXT, LABEL, root='data')
train_data, valid_data = train_data.split()

In [3]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE, 
                 vectors="glove.6B.100d", vectors_cache="vector_cache", 
                 unk_init=torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [4]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE, device=device)

In [5]:
for batch in train_iterator:
    break
text, text_lens = batch.text
print(text)
print(text_lens)

tensor([[  797,     3,    11,  ...,     1,     1,     1],
        [  263,    57,    31,  ...,     1,     1,     1],
        [   25,   325,    73,  ...,     1,     1,     1],
        ...,
        [13416,   195,     3,  ...,     1,     1,     1],
        [   11,   242,    16,  ...,     1,     1,     1],
        [ 1243,    83,    36,  ...,     1,     1,     1]], device='cuda:0')
tensor([731, 494, 274, 221, 682, 362,  92,  61, 284, 420, 269, 576, 246, 161,
        226, 699, 201,  57, 128, 693, 595, 178, 498, 347, 148, 139, 277, 435,
        228, 124, 803, 135, 167, 417, 346, 197, 179,  50, 143, 295, 236, 176,
        241, 236, 172, 284, 305, 513, 548, 111, 214, 132, 121, 217, 286, 306,
        936, 204, 208, 181,  80, 211, 181, 405], device='cuda:0')


# Build Model

In [6]:
class Classifier2D(nn.Module):
    def __init__(self, in_dim, emb_dim, n_filters, filter_sizes, out_dim, dropout, pad_idx):
        super().__init__()
        self.emb = nn.Embedding(in_dim, emb_dim, padding_idx=pad_idx)
        # (1) Original implementation: 2D convolution.
        # `nn.Conv2d(in_channels, out_channels, kernel_size=(H, W))`
        self.convs = nn.ModuleList([
            nn.Conv2d(1, n_filters, kernel_size=(fs, emb_dim)) 
            for fs in filter_sizes])

        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(n_filters*len(filter_sizes), out_dim)

    def forward(self, text, text_lens):
        # `text_lens` would be useful when mean-pooling applied. 
        # text/mask: (batch, step)
        mask = text.eq(self.emb.padding_idx)
        # embedded: (batch, step, emb)
        embedded = self.emb(text)
        
        pooled = []
        for conv_i in self.convs:
            # Note: The kernel size may exceed the sequence length, which leads to invalid output. 
            # conved_i: (batch, channel=n_filters, step-(kernel_size-1), 1)
            conved_i = F.relu(conv_i(embedded.unsqueeze(1)))
            # mask_i: (batch, 1, step-(kernel_size-1))
            mask_i = mask[:, (conv_i.kernel_size[0]-1):].unsqueeze(1)
            # pooled_i: (batch, n_filters)
            pooled_i, _ = conved_i.squeeze(-1).masked_fill(mask_i, -np.inf).max(dim=-1)
            pooled.append(pooled_i)
            
        concated = self.dropout(torch.cat(pooled, dim=-1))
        return self.fc(concated)

In [7]:
class Classifier1D(nn.Module):
    def __init__(self, in_dim, emb_dim, n_filters, filter_sizes, out_dim, dropout, pad_idx):
        super().__init__()
        self.emb = nn.Embedding(in_dim, emb_dim, padding_idx=pad_idx)
        # (2) Regard different embedding dimensions as different channels. 
        # Thus, it reduces to an 1D convolution. 
        self.convs = nn.ModuleList([
            nn.Conv1d(emb_dim, n_filters, kernel_size=fs)
            for fs in filter_sizes])

        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(n_filters*len(filter_sizes), out_dim)

    def forward(self, text, text_lens):
        # `text_lens` would be useful when mean-pooling applied. 
        # text/mask: (batch, step)
        mask = text.eq(self.emb.padding_idx)
        # embedded: (batch, step, emb)
        embedded = self.emb(text)
        
        pooled = []
        for conv_i in self.convs:
            # Note: The kernel size may exceed the sequence length, which leads to invalid output. 
            # conved_i: (batch, channel=n_filters, step-(kernel_size-1))
            conved_i = F.relu(conv_i(embedded.permute(0, 2, 1)))
            # mask_i: (batch, 1, step-(kernel_size-1))
            mask_i = mask[:, (conv_i.kernel_size[0]-1):].unsqueeze(1)
            # pooled_i: (batch, n_filters)
            pooled_i, _ = conved_i.masked_fill(mask_i, -np.inf).max(dim=-1)
            pooled.append(pooled_i)
            
        concated = self.dropout(torch.cat(pooled, dim=-1))
        return self.fc(concated)

In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

IN_DIM = len(TEXT.vocab)
EMB_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3, 4, 5]
OUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

classifier2d = Classifier2D(IN_DIM, EMB_DIM, N_FILTERS, FILTER_SIZES, OUT_DIM, 
                           DROPOUT, PAD_IDX).to(device)
print(count_parameters(classifier2d))

classifier1d = Classifier1D(IN_DIM, EMB_DIM, N_FILTERS, FILTER_SIZES, OUT_DIM, 
                           DROPOUT, PAD_IDX).to(device)
print(count_parameters(classifier1d))

# The two models are identical...
classifier = classifier2d

2620801
2620801


In [9]:
# Initialize Embeddings with Pre-Trained Vectors
classifier.emb.weight.data.copy_(TEXT.vocab.vectors)

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

classifier.emb.weight.data[UNK_IDX].zero_()
classifier.emb.weight.data[PAD_IDX].zero_()

print(classifier.emb.weight[:5, :8])

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281, -0.3996,  0.0832,  0.0440, -0.3914,  0.3344],
        [-0.1077,  0.1105,  0.5981, -0.5436,  0.6740,  0.1066,  0.0389,  0.3548],
        [-0.3398,  0.2094,  0.4635, -0.6479, -0.3838,  0.0380,  0.1713,  0.1598]],
       device='cuda:0', grad_fn=<SliceBackward>)


# Train Model

In [10]:
optimizer = optim.Adam(classifier.parameters())
# Binary cross entropy with logits. 
# The binary version of cross entropy loss. 
loss_func = nn.BCEWithLogitsLoss().to(device)

In [11]:
def train_epoch(classifier, iterator, optimizer, loss_func):
    classifier.train()
    epoch_loss = 0
    epoch_acc = 0
    for batch in iterator:
        # Forward pass
        text, text_lens = batch.text
        preds = classifier(text, text_lens).squeeze(-1)
        # Calculate loss
        loss = loss_func(preds, batch.label)
        # Backward propagation
        optimizer.zero_grad()
        loss.backward()
        # Update weights
        optimizer.step()
        # Accumulate loss and acc
        epoch_loss += loss.item()
        epoch_acc += (torch.round(torch.sigmoid(preds)) == batch.label).sum().item() / preds.size(0)
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

def eval_epoch(classifier, iterator, loss_func):
    classifier.eval()
    epoch_loss = 0
    epoch_acc = 0
    with torch.no_grad():
        for batch in iterator:
            # Forward pass
            text, text_lens = batch.text
            preds = classifier(text, text_lens).squeeze(-1)
            # Calculate loss
            loss = loss_func(preds, batch.label)
            # Accumulate loss and acc
            epoch_loss += loss.item()
            epoch_acc += (torch.round(torch.sigmoid(preds)) == batch.label).sum().item() / preds.size(0)
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

In [12]:
import time
N_EPOCHS = 5
best_valid_loss = np.inf

for epoch in range(N_EPOCHS):
    t0 = time.time()
    train_loss, train_acc = train_epoch(classifier, train_iterator, optimizer, loss_func)
    valid_loss, valid_acc = eval_epoch(classifier, valid_iterator, loss_func)
    epoch_secs = time.time() - t0

    epoch_mins, epoch_secs = int(epoch_secs // 60), int(epoch_secs % 60)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(classifier.state_dict(), 'models/tut4-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 18s
	Train Loss: 0.642 | Train Acc: 62.38%
	 Val. Loss: 0.479 |  Val. Acc: 79.76%
Epoch: 02 | Epoch Time: 0m 18s
	Train Loss: 0.427 | Train Acc: 80.78%
	 Val. Loss: 0.343 |  Val. Acc: 85.14%
Epoch: 03 | Epoch Time: 0m 18s
	Train Loss: 0.305 | Train Acc: 87.25%
	 Val. Loss: 0.308 |  Val. Acc: 86.61%
Epoch: 04 | Epoch Time: 0m 18s
	Train Loss: 0.224 | Train Acc: 91.11%
	 Val. Loss: 0.301 |  Val. Acc: 87.34%
Epoch: 05 | Epoch Time: 0m 18s
	Train Loss: 0.162 | Train Acc: 93.98%
	 Val. Loss: 0.301 |  Val. Acc: 87.67%


In [13]:
classifier.load_state_dict(torch.load('models/tut4-model.pt'))

valid_loss, valid_acc = eval_epoch(classifier, valid_iterator, loss_func)
test_loss, test_acc = eval_epoch(classifier, test_iterator, loss_func)

print(f'Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}%')
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Val. Loss: 0.301 | Val. Acc: 87.34%
Test Loss: 0.347 | Test Acc: 85.16%


# User Input

In [14]:
import spacy
nlp = spacy.load('en')

def predict_senti(classifier, sentence):
    classifier.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[tok] for tok in tokenized]
    lens = len(indexed)

    # Note: `batch_first=True`
    indexed = torch.tensor(indexed, dtype=torch.long).unsqueeze(0).to(device)
    lens = torch.tensor(lens, dtype=torch.long).unsqueeze(0).to(device)
    pred = torch.sigmoid(classifier(indexed, lens)).round().type(torch.long)
    return LABEL.vocab.itos[pred.item()]

In [15]:
predict_senti(classifier, "This is a good film.")

'pos'

In [16]:
predict_senti(classifier, "This film is terrible.")

'neg'

# Check Embeddings
* The Embeddings of `unk` and `<pad>` tokens
    * Because the `padding_idx` has been passed to `nn.Embedding`, so the `<pad>` embedding will remain zeros throughout training.  
    * While the `<unk>` embedding will be learned.

In [17]:
print(classifier.emb.weight[:5, :8])

tensor([[ 0.0399, -0.0643,  0.1684, -0.1979, -0.0529,  0.1593,  0.0633,  0.0702],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0794, -0.2940,  0.7358, -0.4097,  0.0855,  0.1057, -0.4368,  0.3295],
        [-0.0859, -0.0032,  0.6104, -0.5729,  0.6998,  0.1318,  0.0836,  0.4181],
        [-0.2305,  0.0821,  0.4244, -0.6412, -0.3578,  0.0367,  0.1435,  0.0484]],
       device='cuda:0', grad_fn=<SliceBackward>)
